In [1]:
import torch
import torch.nn as nn

In [2]:
# Now let use build our model

In [3]:
# First we need to create a new class- (class name can be of our choice)
class AlexNet(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        # now this is where we start building our main layers
        # One thing we can do here is assign our layers to variables

        # This is our convolutional layer
        self.conv1 = nn.Conv2d(input, 96, kernel_size= 11, stride=4, padding=0)
        # here first we give our input value, then output, kernel size, stride value, and lastly padding value
        # we will follow the same thing in the next layer as well


        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)

        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)



        # now lets build our fully connected layers
        # the settings are the same (kinda)
        self.fcn1= nn.Linear(256 * 6 * 6, 4096)
        self.fcn2 = nn.Linear(4096, 4096)
        self.fcn3 = nn.Linear(4096, output)


        # now lets build activation, pooling, and flatten layers

        self.flatten = nn.Flatten()
        self.maxpooling = nn.MaxPool2d(kernel_size=3, stride =2)
        self.norm = nn.LocalResponseNorm(size=5, k=2)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self._init_weights()

    # now lets build our forward pass
    def forward(self, x):
        x = self.maxpooling(self.norm(self.activation(self.conv1(x))))
        x = self.maxpooling(self.norm(self.activation(self.conv2(x))))
        x = (self.activation(self.conv3(x)))
        x = (self.activation(self.conv4(x)))
        x = self.maxpooling(self.activation(self.conv5(x)))
        x = self.flatten(x)
        x = self.activation(self.fcn1(x))
        x = self.dropout(x)
        x = self.activation(self.fcn2(x))
        x = self.dropout(x)
        return self.fcn3(x)

    # def weight_init(self):
    #     print(next(self.modules()))

    def _init_weights(self):
        for i, layer in enumerate(self.modules()):
            if isinstance(layer, (nn.Conv2d, nn.Linear)):
                nn.init.normal_(layer.weight, mean=0, std=0.01)
                if layer.bias is not None:
                    if i in [1,2,3,4,5,6,7]:  # manual bias 1 if needed
                        nn.init.constant_(layer.bias, val=1)
                    else:
                        nn.init.constant_(layer.bias, val=0)

model = AlexNet(input=3, output=1000)
print(model)

# if __name__ == "__main__":
#     model = AlexNet(input=3, output=1000)
#     print(model(torch.rand(1,3,227,227)).shape)



AlexNet(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fcn1): Linear(in_features=9216, out_features=4096, bias=True)
  (fcn2): Linear(in_features=4096, out_features=4096, bias=True)
  (fcn3): Linear(in_features=4096, out_features=1000, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (maxpooling): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (norm): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
  (activation): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)


In [4]:
!pip install torchinfo



In [5]:
from torchinfo import summary
model = AlexNet(input=3, output=1000)
summary(model, input_size=(1, 3, 227, 227))

Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [1, 1000]                 --
├─Conv2d: 1-1                            [1, 96, 55, 55]           34,944
├─ReLU: 1-2                              [1, 96, 55, 55]           --
├─LocalResponseNorm: 1-3                 [1, 96, 55, 55]           --
├─MaxPool2d: 1-4                         [1, 96, 27, 27]           --
├─Conv2d: 1-5                            [1, 256, 27, 27]          614,656
├─ReLU: 1-6                              [1, 256, 27, 27]          --
├─LocalResponseNorm: 1-7                 [1, 256, 27, 27]          --
├─MaxPool2d: 1-8                         [1, 256, 13, 13]          --
├─Conv2d: 1-9                            [1, 384, 13, 13]          885,120
├─ReLU: 1-10                             [1, 384, 13, 13]          --
├─Conv2d: 1-11                           [1, 384, 13, 13]          1,327,488
├─ReLU: 1-12                             [1, 384, 13, 13]       